#### Preprocessing
Write the dahiti information into geopandas dataframe.

In [183]:
from glob import glob
import geopandas as gpd
from shapely.geometry import Point


In [184]:
dir_data = "data/water-height/hydroweb/hydroweb_lakes"
path_lake = 'data/water-extent/lakes/HydroLakes_v10_hma_region.gpkg'

In [185]:
hma_lakes_gdf = gpd.read_file(path_lake)
hma_lakes_gdf.head(2)


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
0,131,Alakol,Kazakhstan,Asia,SWBD,1,0,2919.34,567.40,2.96,...,1,22.1,55.947,13347.0,347,-1.0,63248.1,81.672917,46.152083,"POLYGON ((81.54844 46.50765, 81.55058 46.50721..."
1,136,Issyk Kul,Kyrgyzstan,Asia,SWBD,1,0,6195.93,608.84,2.18,...,1,280.5,72.678,276778.9,1601,-1.0,21917.0,77.427083,42.485417,"POLYGON ((78.21662 42.7446, 78.21667 42.7421, ..."


In [186]:
lake_id, lake_name, lake_contry, lake_basin, lake_lat, lake_lon = [], [], [], [], [], []
paths_file = glob(dir_data+'/*.txt')
for path_file in paths_file:
    with open(path_file, 'r') as f:
        content = f.read()
        # Extract lake information
        lake_info = {}
        line_first = content.split('\n')[0]
        parts = line_first.split(';')
        lake_id.append(parts[10].split('=')[1].strip())
        lake_name.append(parts[0].split('=')[1])
        lake_contry.append(parts[1].split('=')[1])
        lake_basin.append(parts[2].split('=')[1])
        lake_lat.append(float(parts[3].split('=')[1]))  
        lake_lon.append(float(parts[4].split('=')[1]))

lake_info = {'hydroweb_id': lake_id, 'name': lake_name, 'contry': lake_contry, 'basin': lake_basin}

In [187]:
loc = [Point(lon, lat) for lon, lat in zip(lake_lon, lake_lat)]
hylak_id, hylak_geoms = [], []
for point in loc:
    id_lake = hma_lakes_gdf.geometry.contains(point)    
    if not id_lake.any(): 
        hylak_geoms.append(None)
        hylak_id.append(None)
    else:
        hylak_geoms.append(hma_lakes_gdf.geometry[id_lake].values[0])
        hylak_id.append(hma_lakes_gdf['Hylak_id'][id_lake].values.item())


In [188]:
lake_info.update({'location':loc, 'Hylak_id': hylak_id, 'Hylak_geom': hylak_geoms})


In [191]:
lake_hydroweb_gdf = gpd.GeoDataFrame(lake_info, geometry='location', crs='EPSG:4326')
lake_hydroweb_gdf.head(2)
# lake_hydroweb_gdf.to_file('data/water-height/hydroweb/hydroweb_stations_hma.gpkg', driver='GPKG')


,hydroweb_id,name,contry,basin,location,Hylak_id,Hylak_geom
0,1300000001404,migriggyangzham,China,Mongolian,POINT (90.33 33.45),1404.0,"POLYGON ((90.33479712592299 33.66377685546871,..."
1,1300000001336,hala,China,Mongolian,POINT (97.6 38.3),1336.0,"POLYGON ((97.49625840928849 38.41372219509566,..."
